In [1]:
import tensorflow as tf
import math
from sklearn import datasets
from sklearn.manifold import TSNE
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
#from mpl_toolkits.mplot3d import Axes3D
#import seaborn as sns
%matplotlib notebook
import pymysql
import csv
from scipy.sparse import *
import datetime
#from dbutils_reviewed import *
from scipy.stats import zscore


#from dbutils_copy1 import *
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from nltk.cluster import KMeansClusterer, cosine_distance
from sklearn.cluster import MeanShift, estimate_bandwidth
import pandas as pd
from sklearn.cluster import KMeans
import pickle
import sys 

from sklearn.model_selection import train_test_split # to split the data into two parts
#from sklearn.cross_validation import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
import keras
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        #print "Elapsed time is "+str(time.time()-startTime_for_tictoc)+" seconds."
        print (time.time()-startTime_for_tictoc)
        
import pandas as pd 
import numpy as np 
import os
from scipy.stats import zscore
import math
foldername = "/project/ub-per/MarchData/"
filelist = sorted(os.listdir(foldername))
import pandas, sys, os
folder_list = []
with open("dataframes/foldernames.txt") as p2:
    for line in p2: 
        folder_list.append(line[:-1])
print len(folder_list) 

def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()
def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print "Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds."
    else:
        print "Toc: start time not set"

7


Using TensorFlow backend.


In [2]:
def dataset_resize(dframe, num): 
    table = dframe
    #print table.shape, num ,table.shape[0]-num, num, table.shape[1]
    dataX = np.zeros((table.shape[0]-num,num,table.shape[1]))
    dataY = np.zeros((table.shape[0]-num,table.shape[1]))
    for i in range(len(table)-num):#print table[i:i+num, :].shape , table[i+num-1, :].shape 
        dataX[i,:,:] = table[i:i+num, :]#table[i:i+num, :] 
        dataY[i,:] = table[i+num, :]#print i ,sum(sum(dataX[i,:,:]- dataX[i-1,:,:]))
    return dataX, dataY

In [6]:
from keras.layers import Activation, TimeDistributed,AveragePooling1D
model = Sequential()
model.add(LSTM(32,activation='relu', stateful=False, return_sequences=True,
               input_shape=(32,12))) # 32 timesteps, and 23 features 1
#model.add(Flatten(input_shape=(32,12)))

model.add(TimeDistributed(Dense(1)))

model.add(AveragePooling1D())

model.add(Flatten())

model.add(Dense(12, activation='relu'))

#model.add(Dense(32))
#model.add(Activation('relu'))
model.compile(optimizer = "adam",loss ="mean_squared_error", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32, 32)            5760      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 32, 1)             33        
_________________________________________________________________
average_pooling1d_4 (Average (None, 16, 1)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 12)                204       
Total params: 5,997
Trainable params: 5,997
Non-trainable params: 0
_________________________________________________________________


In [7]:
temp = []
tfile1 = np.array([]) 
for k in range(0,1):#len(folder_list)):
    machine_list = [] 
    with open("filenames/"+folder_list[k]) as p1: 
        for line in p1 : 
            machine_list.append(line[:-1])
    cpu_count = int(folder_list[k].split("cpu")[0])
    for ff in range(0,len(machine_list)):
        tic()
        name_machine = machine_list[ff]
        if os.path.isfile("/project/ub-per/refinedFeb/"+name_machine+".csv") == False:
            continue
        data_frame= pd.read_csv("/project/ub-per/refinedFeb/"+name_machine+".csv")
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^Unnamed')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.read.direct')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.write.direct')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^disk.dev.read_bytes')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^gpfs.fsios')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.read.server')]
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^nfsclient")]# - data_frame.mean(axis = 1 ))#/(data_frame.max(axis=1)-data_frame.min(axis=1))
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^kernel.percpu.cpu.nice")]
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^kernel.percpu.cpu.irq.hard")]
        data_frame = data_frame.iloc[:,1:]
        #data_frame = data_frame.loc[:, data_frame.columns.str.contains('^kernel')]
        #data_frame = data_frame.iloc[:,1:]
        minval = min(data_frame.shape[0], 632) 
        print data_frame.shape 
        #if minval <= 1280:
        #    continue
        data_frame = data_frame.iloc[:minval, :]     
        #print data_frame.shape 
        table = data_frame.apply(zscore, axis = 1 )
        table1 = np.zeros((632,12))
        if len(table) < 632: 
            continue
        for i in range (632): 
            table1[i,:]= (table[i])
        [dataX, datay]=dataset_resize(table1,32)
        print dataX.shape, datay.shape, ff  
        rsayi = np.random.randint(1,20)
        if rsayi == 9 or rsayi == 5:
            history = model.fit(dataX, datay, batch_size=30 , epochs=25, verbose=1)#,shuffle=False)#validation_split=0.33,
            model.reset_states()
            temp.append(ff)
            tfile1 = np.append(tfile1, history.history['loss'])
            #print(history.history['acc'])
        toc()

(1301, 12)
(600, 32, 12) (600, 12) 0
Elapsed time is 0.198400020599 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 1
Elapsed time is 0.196307897568 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 2
Elapsed time is 0.174736976624 seconds.
(1300, 12)
(600, 32, 12) (600, 12) 3
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
600/600 [==============================] - 1s 2ms/step - loss: 1.0267 - acc: 0.0000e+00
Epoch 2/25
600/600 [==============================] - 0s 713us/step - loss: 1.0008 - acc: 0.0400
Epoch 3/25
600/600 [==============================] - 0s 685us/step - loss: 0.9979 - acc: 0.0400
Epoch 4/25
600/600 [==============================] - 0s 720us/step - loss: 0.9967 - acc: 0.0400
Epoch 5/25
600/600 [==============================] - 0s 702us/step - loss: 0.9954 - acc: 0.0400
Epoch 6/25
600/600 [==============================] - 0s 730us/step - loss: 0.9938 - acc: 0.0400
Epoch 7/25
600/600 [==============================] - 0s 692us/step - loss: 0.9918 - acc: 0.0400
Epo

(600, 32, 12) (600, 12) 40
Elapsed time is 0.164474010468 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 41
Elapsed time is 0.147695064545 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 42
Elapsed time is 0.168418884277 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 43
Elapsed time is 0.145963907242 seconds.
(1299, 12)
(600, 32, 12) (600, 12) 44
Elapsed time is 0.160910129547 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 45
Elapsed time is 0.146650075912 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 46
Elapsed time is 0.163888216019 seconds.
(1302, 12)
(600, 32, 12) (600, 12) 47
Elapsed time is 0.158113002777 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 48
Elapsed time is 0.179242134094 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 49
Elapsed time is 0.192315101624 seconds.
(1157, 12)
(600, 32, 12) (600, 12) 50
Epoch 1/25
600/600 [==============================] - 0s 726us/step - loss: 0.1325 - acc: 0.9617
Epoch 2/25
600/600 [==============================] - 0s 704us/step - loss: 0.1318 - acc: 0.9617

600/600 [==============================] - 0s 673us/step - loss: 0.1339 - acc: 0.9567
Epoch 4/25
600/600 [==============================] - 0s 665us/step - loss: 0.1335 - acc: 0.9567
Epoch 5/25
600/600 [==============================] - 0s 688us/step - loss: 0.1332 - acc: 0.9567
Epoch 6/25
600/600 [==============================] - 0s 667us/step - loss: 0.1331 - acc: 0.9567
Epoch 7/25
600/600 [==============================] - 0s 696us/step - loss: 0.1327 - acc: 0.9567
Epoch 8/25
600/600 [==============================] - 0s 660us/step - loss: 0.1326 - acc: 0.9567
Epoch 9/25
600/600 [==============================] - 0s 663us/step - loss: 0.1323 - acc: 0.9567
Epoch 10/25
600/600 [==============================] - 0s 663us/step - loss: 0.1320 - acc: 0.9567
Epoch 11/25
600/600 [==============================] - 0s 660us/step - loss: 0.1318 - acc: 0.9567
Epoch 12/25
600/600 [==============================] - 0s 645us/step - loss: 0.1318 - acc: 0.9567
Epoch 13/25
600/600 [=================

Epoch 25/25
600/600 [==============================] - 0s 690us/step - loss: 0.1227 - acc: 0.9667
Elapsed time is 10.3111169338 seconds.
(1300, 12)
(600, 32, 12) (600, 12) 108
Epoch 1/25
600/600 [==============================] - 0s 740us/step - loss: 0.1974 - acc: 0.9667
Epoch 2/25
600/600 [==============================] - 0s 744us/step - loss: 0.1891 - acc: 0.9667
Epoch 3/25
600/600 [==============================] - 0s 699us/step - loss: 0.1876 - acc: 0.9667
Epoch 4/25
600/600 [==============================] - 0s 681us/step - loss: 0.1869 - acc: 0.9667
Epoch 5/25
600/600 [==============================] - 0s 696us/step - loss: 0.1865 - acc: 0.9667
Epoch 6/25
600/600 [==============================] - 0s 671us/step - loss: 0.1862 - acc: 0.9667
Epoch 7/25
600/600 [==============================] - 0s 679us/step - loss: 0.1861 - acc: 0.9667
Epoch 8/25
600/600 [==============================] - 0s 668us/step - loss: 0.1859 - acc: 0.9667
Epoch 9/25
600/600 [============================

(600, 32, 12) (600, 12) 163
Elapsed time is 0.170116901398 seconds.
(1252, 12)
(600, 32, 12) (600, 12) 164
Elapsed time is 0.150760889053 seconds.
(1300, 12)
(600, 32, 12) (600, 12) 165
Elapsed time is 0.165297031403 seconds.
(1300, 12)
(600, 32, 12) (600, 12) 166
Elapsed time is 0.162590026855 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 167
Elapsed time is 0.172739982605 seconds.
(1302, 12)
(600, 32, 12) (600, 12) 168
Elapsed time is 0.155250072479 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 169
Elapsed time is 0.169105052948 seconds.
(1302, 12)
(600, 32, 12) (600, 12) 170
Elapsed time is 0.165966033936 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 171
Elapsed time is 0.168013811111 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 172
Elapsed time is 0.153136014938 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 173
Elapsed time is 0.16505408287 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 174
Elapsed time is 0.156394004822 seconds.
(1439, 12)
(600, 32, 12) (600, 12) 175
Elapsed time is 0.1739091

600/600 [==============================] - 0s 747us/step - loss: 0.3615 - acc: 0.7067
Epoch 11/25
600/600 [==============================] - 0s 771us/step - loss: 0.3616 - acc: 0.7050
Epoch 12/25
600/600 [==============================] - 0s 685us/step - loss: 0.3612 - acc: 0.7050
Epoch 13/25
600/600 [==============================] - 0s 661us/step - loss: 0.3610 - acc: 0.7050
Epoch 14/25
600/600 [==============================] - 0s 673us/step - loss: 0.3610 - acc: 0.7050
Epoch 15/25
600/600 [==============================] - 0s 670us/step - loss: 0.3609 - acc: 0.7067
Epoch 16/25
600/600 [==============================] - 0s 751us/step - loss: 0.3607 - acc: 0.7067
Epoch 17/25
600/600 [==============================] - 0s 751us/step - loss: 0.3607 - acc: 0.7067
Epoch 18/25
600/600 [==============================] - 0s 762us/step - loss: 0.3604 - acc: 0.7067
Epoch 19/25
600/600 [==============================] - 0s 745us/step - loss: 0.3606 - acc: 0.7050
Epoch 20/25
600/600 [===========

600/600 [==============================] - 0s 678us/step - loss: 0.1130 - acc: 0.9783
Elapsed time is 10.6258649826 seconds.
(1302, 12)
(600, 32, 12) (600, 12) 229
Elapsed time is 0.178767204285 seconds.
(1221, 12)
(600, 32, 12) (600, 12) 230
Elapsed time is 0.171672105789 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 231
Elapsed time is 0.159703969955 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 232
Epoch 1/25
600/600 [==============================] - 0s 720us/step - loss: 0.3417 - acc: 0.7217
Epoch 2/25
600/600 [==============================] - 0s 652us/step - loss: 0.3364 - acc: 0.7300
Epoch 3/25
600/600 [==============================] - 0s 671us/step - loss: 0.3336 - acc: 0.7283
Epoch 4/25
600/600 [==============================] - 0s 678us/step - loss: 0.3306 - acc: 0.7367
Epoch 5/25
600/600 [==============================] - 0s 673us/step - loss: 0.3313 - acc: 0.7317
Epoch 6/25
600/600 [==============================] - 0s 685us/step - loss: 0.3299 - acc: 0.7333
Epoch 7/25
600/60

600/600 [==============================] - 0s 750us/step - loss: 0.1049 - acc: 0.9733
Epoch 14/25
600/600 [==============================] - 0s 778us/step - loss: 0.1033 - acc: 0.9783
Epoch 15/25
600/600 [==============================] - 0s 730us/step - loss: 0.1068 - acc: 0.9700
Epoch 16/25
600/600 [==============================] - 0s 661us/step - loss: 0.1039 - acc: 0.9750
Epoch 17/25
600/600 [==============================] - 0s 680us/step - loss: 0.1028 - acc: 0.9750
Epoch 18/25
600/600 [==============================] - 0s 686us/step - loss: 0.1036 - acc: 0.9767
Epoch 19/25
600/600 [==============================] - 0s 655us/step - loss: 0.1374 - acc: 0.9550
Epoch 20/25
600/600 [==============================] - 0s 679us/step - loss: 0.1347 - acc: 0.9550
Epoch 21/25
600/600 [==============================] - 0s 671us/step - loss: 0.1335 - acc: 0.9550
Epoch 22/25
600/600 [==============================] - 0s 667us/step - loss: 0.1329 - acc: 0.9550
Epoch 23/25
600/600 [===========

(600, 32, 12) (600, 12) 292
Elapsed time is 0.167464971542 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 293
Elapsed time is 0.1626060009 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 294
Elapsed time is 0.163904905319 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 295
Elapsed time is 0.160844802856 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 296
Elapsed time is 0.157783985138 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 297
Elapsed time is 0.152781963348 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 298
Elapsed time is 0.15255689621 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 299
Elapsed time is 0.159981966019 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 300
Elapsed time is 0.153675794601 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 301
Elapsed time is 0.149298906326 seconds.
(1302, 12)
(600, 32, 12) (600, 12) 302
Elapsed time is 0.150332927704 seconds.
(1301, 12)
(600, 32, 12) (600, 12) 303
Elapsed time is 0.149715900421 seconds.
(1438, 12)
(600, 32, 12) (600, 12) 304
Elapsed time is 0.157419919

In [9]:
from keras.models import model_from_json, load_model
model.save("models/June3_smoothModel_e100.h5")

In [10]:
myt = tfile1.reshape([29,100])
plt.ion()
for i in range (29): 
    plt.figure()
    #print myt[i]
    plt.plot(myt[i])
    

ValueError: cannot reshape array of size 800 into shape (29,100)

In [ ]:
import pickle
pickle.dump(tfile1, open("tfile1.p", "w"))

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
testing_set = np.array(list(set(range(0,329))-set(temp[:327])))
testing_set

In [11]:
nn = 2400
lenw = 32
testingset = 300
prediction = np.zeros(((testingset)-32, nn-lenw, 12))
prediction_diff = np.zeros(((testingset)-32,  nn-lenw, 12))
dataylist = np.zeros(((testingset)-32,  nn-lenw, 12))
prediction1 = np.zeros(((testingset)-32,  nn-lenw, 12))
prediction_diff1 = np.zeros(((testingset)-32,  nn-lenw, 12))
dataylist1 = np.zeros(((testingset)-32,  nn-lenw, 12))

In [ ]:
from keras.models import model_from_json, load_model

In [ ]:
#model.save("model_result_all_metrics_epoch100_20perc.h5")

In [12]:
import math 
for k in range(0,1):#len(folder_list)):
    machine_list = [] 
    with open("filenames/"+folder_list[k]) as p1: 
        for line in p1 : 
            machine_list.append(line[:-1])
    cpu_count = int(folder_list[k].split("cpu")[0])
    cnt = 0 
    for ff in range(325):#testing_set:
        tic()
        name_machine = machine_list[ff]
        print ff 
        if os.path.isfile("/project/ub-per/refinedMarch/"+name_machine+".csv") == False:
            continue
        data_frame= pd.read_csv("/project/ub-per/refinedMarch/"+name_machine+".csv")
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^Unnamed')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.read.direct')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.write.direct')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^disk.dev.read_bytes')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^gpfs.fsios')]
        data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^nfsclient.bytes.read.server')]
        #data_frame = data_frame.loc[:, data_frame.columns.str.contains('^kernel')]
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^nfsclient")]# - data_frame.mean(axis = 1 ))#/(data_frame.max(axis=1)-data_frame.min(axis=1))
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^kernel.percpu.cpu.nice")]
        data_frame = data_frame.loc[:,~data_frame.columns.str.contains("^kernel.percpu.cpu.irq.hard")]
        data_frame = data_frame.iloc[:,1:]
        if data_frame.shape[0] < 1300:
            continue
        #data_frame = data_frame.loc[:, ~data_frame.columns.str.contains('^Unnamed')]
        #data_frame = data_frame.loc[:, data_frame.columns.str.contains('^kernel.percpu.cpu')]
        minval = min(data_frame.shape[0], 2400) 
        data_frame = data_frame.iloc[:minval, :]#+math.pow(10,-10)
        table = data_frame.apply(zscore, axis = 1)
        
        #table = (data_frame - data_frame.mean(axis=1))/data_frame.std(ddof=0)
        if table.shape[0]==2400:
            em = np.zeros((2400, 12))
            table1 = np.zeros((2400, 12))
            for i in range (2400): 
                em[i,:]= (table[i])
            table = em 
            for u in range(2400):
                if u > 1800 and u< 1900: 
                    table1[u] = table[u]+2
                else : 
                    table1[u] = table[u]
                    

            [datax, datay]=dataset_resize(table, 32)
            prediction[cnt,:,:]=(model.predict(datax))
            prediction_diff[cnt,:,:] = prediction[cnt,:,:]-datay
            dataylist[cnt,:,:] = datay
            
            [datax1, datay1]=dataset_resize(table1, 32)
            prediction1[cnt,:,:]=(model.predict(datax1))
            prediction_diff1[cnt,:,:] = prediction1[cnt,:,:]-datay1
            dataylist1[cnt,:,:] = datay1
            cnt+=1
        toc()

0
Elapsed time is 1.39619994164 seconds.
1
Elapsed time is 1.16915297508 seconds.
2
3
Elapsed time is 1.23848485947 seconds.
4
Elapsed time is 1.20983505249 seconds.
5
Elapsed time is 1.21542692184 seconds.
6
Elapsed time is 1.18390607834 seconds.
7
Elapsed time is 1.2298719883 seconds.
8
Elapsed time is 1.21612691879 seconds.
9
Elapsed time is 1.38156294823 seconds.
10
Elapsed time is 1.26052498817 seconds.
11
Elapsed time is 1.31029415131 seconds.
12
Elapsed time is 1.18877696991 seconds.
13
Elapsed time is 1.14359807968 seconds.
14
Elapsed time is 1.27279901505 seconds.
15
Elapsed time is 1.26981115341 seconds.
16
17
Elapsed time is 1.23756599426 seconds.
18
Elapsed time is 1.19964599609 seconds.
19
Elapsed time is 1.16526508331 seconds.
20
Elapsed time is 1.26692199707 seconds.
21
Elapsed time is 1.21332812309 seconds.
22
Elapsed time is 1.24466490746 seconds.
23
Elapsed time is 1.30731391907 seconds.
24
Elapsed time is 1.42126893997 seconds.
25
Elapsed time is 1.27641415596 second

Elapsed time is 1.32319688797 seconds.
227
228
Elapsed time is 1.29606485367 seconds.
229
Elapsed time is 1.2613940239 seconds.
230
Elapsed time is 1.23417115211 seconds.
231
Elapsed time is 1.19629001617 seconds.
232
Elapsed time is 1.24581980705 seconds.
233
Elapsed time is 1.2050101757 seconds.
234
Elapsed time is 1.23212194443 seconds.
235
236
Elapsed time is 1.19556689262 seconds.
237
238
239
Elapsed time is 1.24901580811 seconds.
240
Elapsed time is 1.15256309509 seconds.
241
Elapsed time is 1.20419311523 seconds.
242
Elapsed time is 1.21258497238 seconds.
243
Elapsed time is 1.17621803284 seconds.
244
Elapsed time is 1.26668095589 seconds.
245
Elapsed time is 1.36466407776 seconds.
246
Elapsed time is 1.21117377281 seconds.
247
Elapsed time is 1.21397185326 seconds.
248
Elapsed time is 1.21886396408 seconds.
249
Elapsed time is 1.17772006989 seconds.
250
Elapsed time is 1.22750902176 seconds.
251
252
253
Elapsed time is 1.15608406067 seconds.
254
Elapsed time is 1.24276995659 se

IndexError: index 268 is out of bounds for axis 0 with size 268

In [ ]:
#prediction_diff[:272,:,:],prediction_diff1[:272,:,:]

In [13]:
temp = np.zeros((prediction_diff.shape[0],prediction_diff.shape[1]))
temp1 = np.zeros((prediction_diff.shape[0],prediction_diff.shape[1]))
for i in range(prediction_diff.shape[0]):
    print i, prediction_diff[i] 
    temp[i,:] = (np.linalg.norm(prediction_diff[i],axis=1))
    temp1[i,:] = (np.linalg.norm(prediction_diff1[i],axis=1))
print temp.shape, temp1.shape 
temp[:272,:], temp1[:272,:]

0 [[ 0.48541766 -2.22207677 -2.22203093 ...  0.07705886  0.48375141
   0.48539789]
 [ 0.48541042 -2.22207393 -2.22202942 ...  0.07702947  0.48388997
   0.4853355 ]
 [ 0.48443009 -2.22284544 -2.22280088 ...  0.08740703  0.48284102
   0.48441165]
 ...
 [ 0.30331667  0.30330549  0.30330549 ... -0.08066858  0.28388224
   0.30312621]
 [ 0.30335186  0.30333964  0.30333964 ... -0.08044308  0.28412741
   0.30257149]
 [ 0.30328291  0.30327066  0.30327066 ... -0.08023071  0.2842168
   0.30310625]]
1 [[ 0.3028653   0.3028553   0.3028553  ...  0.22143072  0.28853688
   0.3025076 ]
 [ 0.30280964  0.30279882  0.30279882 ... -0.10294457  0.2895305
   0.3020252 ]
 [ 0.30286927  0.30285851  0.30285851 ...  0.00858274  0.28842145
   0.30259355]
 ...
 [ 0.30359853  0.30358447  0.30358447 ... -0.06197351  0.28116461
   0.30334416]
 [ 0.30357266  0.30355684  0.30355684 ...  0.03477067  0.28137764
   0.30337664]
 [ 0.30360237  0.30358674  0.30358674 ...  0.05341941  0.28108858
   0.30339685]]
2 [[0.3158047 

 [0.31720327 0.31718885 0.31718885 ... 0.00452514 0.29264062 0.16664639]]
141 [[ 0.32114717  0.32113684  0.32113684 ...  0.00479635  0.10676829
   0.31401893]
 [ 0.31858188  0.31857121  0.31857121 ...  0.02783485  0.13469463
   0.31070755]
 [ 0.32100735  0.32099697  0.32099697 ...  0.04515732  0.1088673
   0.31330817]
 ...
 [ 0.30333036  0.30331677  0.30331677 ... -0.08066357  0.28375066
   0.30315308]
 [ 0.30331679  0.30330321  0.30330321 ... -0.08034167  0.28387637
   0.3031517 ]
 [ 0.30335209  0.30333937  0.30333937 ... -0.08015638  0.2835611
   0.30314066]]
142 [[ 0.48343585 -2.22030531 -2.22684552 ...  0.09784692  0.48186361
   0.48341229]
 [ 0.48293886 -2.22054828 -2.22731337 ...  0.10287646  0.4815152
   0.48292109]
 [ 0.48322542 -2.22002152 -2.22740912 ...  0.09982466  0.48184422
   0.48320719]
 ...
 [ 0.38261162  0.38259984  0.38259984 ...  0.14623311  0.05547425
  -0.23952326]
 [ 0.35855377  0.35854084  0.35854084 ...  0.09492621  0.33695685
  -0.30199454]
 [ 0.35769275  0.35

(array([[3.46410162, 3.46410162, 3.46410162, ..., 1.00340456, 1.00338236,
         1.00336357],
        [1.02430531, 1.00535704, 1.00012322, ..., 1.00212676, 1.00080848,
         1.0016357 ],
        [1.01066312, 1.0140358 , 1.00014261, ..., 1.0092381 , 1.01066005,
         1.00851495],
        ...,
        [3.46410162, 3.46410162, 3.46410162, ..., 1.0095353 , 1.01000939,
         1.01089998],
        [1.01257313, 1.00021941, 1.00009526, ..., 1.01255875, 1.00740652,
         1.01454512],
        [3.46410162, 3.46410162, 3.46410162, ..., 1.00873903, 1.00879215,
         1.00844821]]),
 array([[3.46410162, 3.46410162, 3.46410162, ..., 1.00340456, 1.00338236,
         1.00336357],
        [1.02430531, 1.00535704, 1.00012322, ..., 1.00212676, 1.00080848,
         1.0016357 ],
        [1.01066312, 1.0140358 , 1.00014261, ..., 1.0092381 , 1.01066005,
         1.00851495],
        ...,
        [3.46410162, 3.46410162, 3.46410162, ..., 1.0095353 , 1.01000939,
         1.01089998],
        [1.0

In [ ]:
'''df = data_frame.sub(data_frame.mean(axis=1), axis=0)
#print df.shape 
df1 = data_frame.max(axis = 1 )-data_frame.min(axis = 1 )
#print df1 
df2 =data_frame.std(ddof=0, axis = 1)
#print df2
table = df.apply(zscore, axis = 1 )
table '''

In [14]:
(temp).shape 

(268, 2368)

In [15]:
for i in range (len(temp)-1): 
    if sum(np.isnan(temp[i,:])) > 0 : 
        temp = np.delete(temp, i, 0 )
        i = i -1 


In [16]:
for i in range (len(temp1)-1): 
    if sum(np.isnan(temp1[i,:])) > 0 : 
        temp1 = np.delete(temp1, i, 0 )
        i = i -1 

In [ ]:
#np.array([2,4,6])/np.array([2,2,2])

In [17]:
temp3= np.zeros((272,prediction_diff.shape[1]))
temp4 = np.zeros((272,prediction_diff.shape[1]))
mnval = np.mean(temp[:271,:],axis = 0 )  
mnval1 = np.mean(temp1[:271,:],axis = 0 ) 
for i in range(267): 
    temp3[i,:] = (temp[i,:]/mnval)
    temp4[i,:] = (temp1[i,:]/mnval)

In [18]:
plt.ioff()
from matplotlib.pyplot import figure
for v in range(temp.shape[0]):
    figure(figsize=(12, 9), facecolor='w', edgecolor='k')
    plt.title("Prediction on " +str(v))
    plt.plot(temp3[v])
    plt.plot(temp4[v])
    plt.xlabel("time scale")
    plt.ylabel("prediction discrepency")
    plt.savefig("H3figuresMarch100/figurePred"+"%03d" % (v,)+".pdf")
    

/home/niyaziso/anaconda27/lib/python2.7/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
nantemp =np.append(temp[0:30,:], temp[31:270,:])
#nantemp1 = np.delete(temp1, 30, 0)


In [ ]:
nantemp

In [ ]:
#my1 = np.mean(temp[:272,:],axis=0)
#my2 = np.mean(temp[:272,:],axis=0)
my1 = temp[:272,:]
my2 = temp1[:272,:]
print my1.shape 
my1[0:29,:]
my13 = my1[31:,:]
print my13.shape 
my13 = my13[~np.isnan(my13)]
my13.reshape([241, 2368])

In [ ]:
641728/272.0

In [ ]:
plt.ioff()
from matplotlib.pyplot import figure
for v in range(temp.shape[0]):
    figure(figsize=(12, 9), facecolor='w', edgecolor='k')
    plt.title("Prediction on " +str(v))
    plt.xlabel("time scale")
    plt.ylabel("prediction discrepency")
    plt.savefig("figuresMarch100/figurePred"+"%03d" % (v,)+".pdf")
    plt.plot(temp[v])
    plt.plot(temp1[v])


In [ ]:
plt.ioff()
from matplotlib.pyplot import figure
for v in range(temp.shape[0]):
    figure(figsize=(12, 9), facecolor='w', edgecolor='k')
    plt.plot(temp[v])
    plt.plot(temp1[v])
    plt.title("Prediction on " +str(v))
    plt.xlabel("time scale")
    plt.ylabel("prediction discrepency")
    plt.savefig("figuresMarch100/figurePred"+"%03d" % (v,)+".pdf")
    